# Interacting with the Polaris catalog

You can interact with the Polaris catalog through the [management](https://polaris.apache.org/in-dev/unreleased/polaris-management-service/) and [catalog](https://polaris.apache.org/in-dev/unreleased/rest-catalog-open-api/) APIs.

In [1]:
import httpx

mgmt_client = httpx.Client(
    base_url="http://polaris:8181/api/management/v1/",
    headers={
        "Authorization": "Bearer principal:root;realm:default-realm",
        "Content-Type": "application/json",
    }
)

catalog_client = httpx.Client(
    base_url="http://polaris:8181/api/catalog/v1",
    headers={
        "Authorization": "Bearer principal:root;realm:default-realm",
        "Content-Type": "application/json",
    }
)

For example, we can retrieve the catalogs, namespaces and tables that we've created:

In [41]:
catalogs = mgmt_client.get("catalogs").json()
catalog_name = catalogs["catalogs"][0]["name"]

namespaces = catalog_client.get(f"{catalog_name}/namespaces").json()
namespace_name = namespaces["namespaces"][0][0]

tables = catalog_client.get(f"{catalog_name}/namespaces/{namespace_name}/tables").json()
table_names = [table["name"] for table in tables["identifiers"]]

In [43]:
from asciitree import LeftAligned
from collections import OrderedDict as OD

tree = {
     f"Catalog: {catalog_name}": OD([
         (
             f'Namespace: {namespace_name}', OD([
                 (f'Table: {table}', {}) for table in table_names
             ])
         )
     ])
}

tr = LeftAligned()
print(tr(tree))

Catalog: dagster_example_catalog
 +-- Namespace: air_quality
     +-- Table: daily_air_quality_data
     +-- Table: daily_avg_air_quality_data


And we can retrieve the principal, principal roles, catalog roles, and grants:

In [119]:
principals = [principal["name"] for principal in mgmt_client.get("principals").json()["principals"]]

In [120]:
principals_ = {}

for principal in principals:
    principal_roles = mgmt_client.get(f"principals/{principal}/principal-roles").json()
    principal_roles_ = {}
    for role in principal_roles["roles"]:
        role_name = role["name"]
        catalog_roles = (
            mgmt_client.get(f"principal-roles/{role_name}/catalog-roles/{catalog_name}").json()
        )
        catalog_role_name = [catalog_role["name"] for catalog_role in catalog_roles["roles"]]
        grants = [
            grant["privilege"] 
            for grant in (
                mgmt_client.get(f"catalogs/{catalog_name}/catalog-roles/{catalog_role_name[0]}/grants").json().get("grants", [])
            )
        ]
        principal_roles_[role_name] = {
            "catalog_roles" : catalog_role_name,
            "grants" : grants
        }
    principals_[principal] = principal_roles_
    
print(principals_)

{'root': {'service_admin': {'catalog_roles': ['catalog_admin'], 'grants': ['CATALOG_MANAGE_ACCESS', 'CATALOG_MANAGE_METADATA']}}, 'dagster': {'dagster_principal_role': {'catalog_roles': ['orchestration_catalog_role'], 'grants': ['CATALOG_MANAGE_CONTENT']}}}


In [121]:
tree = []

for principal, roles in principals_.items():
    tree.append(
        (f'Principal: {principal}',
            OD([
            (f'Principal role: {list(roles.keys())[0]}',
            OD([
                (f"Catalog role: {roles[list(roles.keys())[0]]['catalog_roles'][0]}", 
                OD([
                        (f"Grant: {grant}", {}) for grant in roles[list(roles.keys())[0]]["grants"]
                ]))
            ]))
        ])
        )
    )
    
print(tr({"Principals": OD(tree)}))

Principals
 +-- Principal: root
 |   +-- Principal role: service_admin
 |       +-- Catalog role: catalog_admin
 |           +-- Grant: CATALOG_MANAGE_ACCESS
 |           +-- Grant: CATALOG_MANAGE_METADATA
 +-- Principal: dagster
     +-- Principal role: dagster_principal_role
         +-- Catalog role: orchestration_catalog_role
             +-- Grant: CATALOG_MANAGE_CONTENT
